# LLM Tracing with W&B

## 1. Auto-logging

In this section, we will call OpenAI LLM to generate names of our game assets. We will use W&B autologging, also available for other popular LLMs and libraries like ... 

In [1]:
# Install wandb-addons, this will be added to wandb soon
# !git clone https://github.com/soumik12345/wandb-addons.git
# !pip install ./wandb-addons[prompts] openai wandb -qqq

In [2]:
import os
import random
import time
import datetime

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.integration.openai import autolog
from wandb_addons.prompts import Trace

In [3]:
PROJECT = "deeplearningai-llm"

In [4]:
autolog({"project":PROJECT, "job_type": "generation", "anonymous":"allow"})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anony-moose-57595088200203951. Use `wandb login --relogin` to force relogin


In [5]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [6]:
MODEL_NAME = "gpt-3.5-turbo"

In [7]:
def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))

In [8]:
# openai.api_key = os.environ["OPENAI_API_KEY"]  # this probably needs some setup on their platform

In [9]:
system_prompt = """You are a creative copywriter.
You're given a category of game asset, and your goal is to design a name of that asset.
The game is set in a fantasy world where everyone laughs and respects each other, while celebrating diversity."""
user_prompt = "hero"
generate_and_print(system_prompt, user_prompt)

wandb: WARNING Artifacts logged anonymously cannot be claimed and expire after 7 days.


Euphoric Guardian

Harmony's Chosen

"Hilaria: The Laughing Guardian"

Unity's Embrace

Harmonic Defender

In [10]:
user_prompt = "jewel"
generate_and_print(system_prompt, user_prompt)

wandb: WARNING Artifacts logged anonymously cannot be claimed and expire after 7 days.


Harmony Gems

Gleam Haven

Gleaming Harmony Gem

Gleamstone

Harmony Gems

In [11]:
wandb.finish()

usage/completion_tokens,█▁
usage/elapsed_time,█▁
usage/prompt_tokens,▁█
usage/total_tokens,█▁
usage/completion_tokens,19
usage/elapsed_time,0.74012
usage/prompt_tokens,62
usage/total_tokens,81


## 2. Using Tracer to log more complex chains

How can we get more creative outputs? Let's design an LLM chain that will first randomly pick a fantasy world, and then generate character names. We will demonstrate how to use Tracer in such scenario. You can also use our native integration with libraries like Langchain or Llamaindex instead. 

In [12]:
worlds = [
    "a mystic medieval island inhabited by intelligent and funny frogs",
    "a modern castle sitting on top of a volcano in a faraway galaxy",
    "a digital world inhabited by friendly machine learning engineers"
]

In [13]:
random.choice(worlds)

'a mystic medieval island inhabited by intelligent and funny frogs'

In [14]:
# define your conifg
model_name = "gpt-3.5-turbo"
temperature = 0.7
system_message = """You are a creative copywriter. 
You're given a category of game asset, a fantasy world, and your goal is to design a name of that asset.
Provide the resulting name only, no additional description.
Single name, max 3 words output, remember!"""

In [15]:
def run_creative_chain(query):
    # part 1 - a chain is started...
    start_time_ms = round(datetime.datetime.now().timestamp() * 1000)

    root_span = Trace(
          name="MyCreativeChain",
          kind="agent",
          start_time_ms=start_time_ms,
          metadata={"user": "student_1"})

    # part 2 - The chain calls into a child chain..
    chain_span = Trace(
          name="MyChain",
          kind="chain",
          start_time_ms=start_time_ms)

    # add the Chain span as a child of the root
    root_span.add_child(chain_span)

    # part 3 - your chain picks a fantasy world
    time.sleep(3)
    world = random.choice(worlds)
    expanded_prompt = f'Game asset category: {query}; fantasy world description: {world}'
    tool_end_time_ms = round(datetime.datetime.now().timestamp() * 1000)

    # create a Tool span 
    tool_span = Trace(
          name="WorldPicker",
          kind="tool",
          status_code="success",
          start_time_ms=start_time_ms,
          end_time_ms=tool_end_time_ms,
          inputs={"input": query},
          outputs={"result": expanded_prompt})

    # add the TOOL span as a child of the root
    chain_span.add_child(tool_span)

    # part 4 - the LLMChain calls an OpenAI LLM...
    messages=[
      {"role": "system", "content": system_message},
      {"role": "user", "content": expanded_prompt}
    ]

    response = openai.ChatCompletion.create(model=model_name,
                                            messages=messages,
                                            temperature=temperature)   

    llm_end_time_ms = round(datetime.datetime.now().timestamp() * 1000)
    response_text = response["choices"][0]["message"]["content"]
    token_usage = response["usage"].to_dict()

    llm_span = Trace(
          name="OpenAI",
          kind="llm",
          status_code="success",
          metadata={"temperature":temperature,
                    "token_usage": token_usage, 
                    "model_name":model_name},
          start_time_ms=tool_end_time_ms,
          end_time_ms=llm_end_time_ms,
          inputs={"system_prompt":system_message, "query":expanded_prompt},
          outputs={"response": response_text},
          )

    # add the LLM span as a child of the Chain span...
    chain_span.add_child(llm_span)

    # update the end time of the Chain span
    chain_span.add_inputs_and_outputs(
          inputs={"query":query},
          outputs={"response": response_text})

    # update the Chain span's end time
    chain_span._span.end_time_ms = llm_end_time_ms

    # part 5 - the final results from the tool are added 
    root_span.add_inputs_and_outputs(inputs={"query": query},
                                     outputs={"result": response_text})
    root_span._span.end_time_ms = llm_end_time_ms

    # part 6 - log all spans to W&B by logging the root span
    root_span.log(name="trace")
    print(f"Result: {response_text}")


In [16]:
wandb.init(project=PROJECT, job_type="generation", anonymous="allow")

run_creative_chain("hero")

Result: Croakvalor


In [17]:
run_creative_chain("jewel")

Result: Volcanium Gem


In [ ]:
wandb.finish()

## Langchain agent

WIP: add langchain agent - adding names and evaluating if they are good. Wrap a previous function as a langchain tool. 

Demonstrate W&B Tracer autologging. 

## Finish the run